create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to in as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

the job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

In [15]:
#import some libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sqlite3
import datetime

In [16]:
url = "https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attribs = ["Name", "MC_USD_Billion","MC_GBP_Billion","MC_EUR_Billion","MC_INR_Billion"]
db_name = 'bank_exchange.db'
table_name = 'Largest_banks'
csv_path = 'C:/Users/E7440/OneDrive/Desktop/data_engineering/webscraping_and_API/banks_exchange_etl/banks_gdp.csv'

In [17]:
#create a log function 

def log(message):
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfiles.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [18]:
# Call the function here
def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    tables =soup.find_all('tbody')
    rows = tables[0].find_all('tr')

    for row in rows:
        if row.find('td') is not None:
            col = row.find_all('td')
            bank_name = col[1].find_all('a')[1]['title']
            market_cap =col[2].contents[0][:-1]
            data_dict = {'Name':bank_name,
                         'MC:USD_Billion':float(market_cap)}
            df1 =pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
    return df

In [19]:
# Call the function here
def transform(df,csv_path):
    exchange_rate = pd.read_csv(csv_path)
    exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']

    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

    return df


In [20]:
#load the file
def load_to_csv(df, output_path):
    df.to_csv(output_path)

#load the databease
def load_to_db(df,sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


In [21]:
def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [26]:
log('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log('Data extraction complete. Initiating Transformation process')

log('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)
log('Data saved to CSV file')

sql_connection = sqlite3.connect('bank_exchange.db')

log('SQL Connection initiated.')

load_to_db(df, sql_connection,table_name)

log('Data loaded to Database as table. Running the query')

query_statement1 = f"SELECT * FROM Largest_banks"
run_query(query_statement1, sql_connection)

query_statement2 = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement2, sql_connection)

query_statement3 =f"SELECT Name from Largest_banks LIMIT 5"
run_query(query_statement3, sql_connection)

log('Process Complete.')

sql_connection.close()

SELECT * FROM Largest_banks
                                      Name MC_USD_Billion MC_GBP_Billion  \
0                           JPMorgan Chase           None           None   
1                          Bank of America           None           None   
2  Industrial and Commercial Bank of China           None           None   
3                              Wells Fargo           None           None   
4               Agricultural Bank of China           None           None   
5                                HDFC Bank           None           None   
6                                     HSBC           None           None   
7                           Morgan Stanley           None           None   
8                  China Construction Bank           None           None   
9                            Bank of China           None           None   

  MC_EUR_Billion MC_INR_Billion  MC:USD_Billion  
0           None           None          491.76  
1           None           None    